<a target="_blank" href="https://colab.research.google.com/github/PeterS111/Fine-tuning-GPT-3-for-Poetry-Generation-and-Evaluation/blob/main/Code/GPT_3_FINE_TUNING.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

### This notebook fine-tunes GPT-3 models

for details please refer to our paper: https://github.com/PeterS111/Fine-tuning-GPT-3-for-Poetry-Generation-and-Evaluation/blob/main/sawicki2023gptpoetry.pdf

author: PeterS

version: 29/04/2023

In [ ]:
# !pip install openai
import openai

In [ ]:
openai.api_key = "your OpenAI key here"

In [ ]:
model = "babbage" 
n_epochs = 4 
batch_size = 4
learning_rate_multiplier = 0.1
prompt_loss_weight = 0.1
fine_tuning_file_path = 'wordsworth.jsonl'
suffix = fine_tuning_file_path.split(".jsonl")[0] + "_" + model +  "_" + str(n_epochs) +"e"
print(suffix)

In [ ]:
def upload_file_to_openai(file_path):
    with open(file_path, "rb") as f:
        file_data = f.read()

    response = openai.File.create(
        purpose="fine-tune",
        file=file_data
    )
    return response

response = upload_file_to_openai(fine_tuning_file_path)

print("File ID:", response.id)

def fine_tune_gpt3(file_id):
    response = openai.FineTune.create(
        model=model,
        training_file=file_id,
        n_epochs=n_epochs,
        batch_size=batch_size,
        suffix=suffix,
        learning_rate_multiplier=learning_rate_multiplier,
        prompt_loss_weight=prompt_loss_weight        
               
    )
    return response

# Start the fine-tuning process

fine_tune_response = fine_tune_gpt3(response.id)
print("Fine-tuning ID:", fine_tune_response.id)



In [ ]:
# Run this cell to show the progress of fine-tuning:

def follow_fine_tune_progress(fine_tune_id):
    
    f_dict = openai.FineTune.retrieve(fine_tune_id)
    for i in f_dict["events"]:
        print(i["message"])

follow_fine_tune_progress(fine_tune_response.id)


In [ ]:
# Uncomment the last line and run this cell if you want to cancel fine-tuning
# Just closing this notebook will not cancel the fine-tuning!

def cancel_fine_tune(fine_tune_id):
    
    response_dict = openai.FineTune.cancel(fine_tune_id)
    print(response_dict["events"][0]["message"])
    print(response_dict["events"][1]["message"])
    
# cancel_fine_tune(fine_tune_response.id)
